In [52]:
import pyvisa
import time
import numpy as np
import urllib.request #para abrir URLs 
import functools as ft #operações complexas
from bs4 import BeautifulSoup
import re

In [104]:
rm = pyvisa.ResourceManager()
rm.list_resources()

()

In [4]:
try:
    keithley = rm.open_resource('USB0::0x05E6::0x2450::04608397::INSTR') #connecting to device via USB 
except pyvisa.errors.VisaIOError:
    try:
        keithley = rm.open_resource('TCPIP0::10.10.15.107::inst0::INSTR') #connecting to device via LAN
    except pyvisa.errors.VisaIOError:
        keithley = None

print(keithley)

None


In [58]:
identification = keithley.query("*IDN?")
print('\n Identification of the keithley:', identification) #checking identification of the device being used


 Identification of the instrument: KEITHLEY INSTRUMENTS,MODEL 2450,04608397,1.7.12b



In [122]:
t0 = time.time()
keithley.write(':SOUR:FUNC CURR')  # Set the source function to current
keithley.write('SOUR:CURR:DEL:AUTO OFF')
current = 25e-9
keithley.write(f':SOUR:CURR {current}')  # Set the source current to 100uA
keithley.write(':OUTP ON')  # Turn on the output
keithley.write('DISPlay:SCReen SWIPE_USER') #activate display text in the instrument
keithley.write(f'DISPlay:USER1:TEXT "CURR: {current} A"') #show the current being sourced. this was done due to display problems
t1 = time.time()

In [123]:
t2=time.time()
keithley.write(':SOUR:CURR?')  # Query the current source value
time.sleep(1)
source_current = keithley.read()  # Read the current source value
print('\n Current value (A): {:.3e}'.format(float(source_current)))
t3 = time.time()


 Current value (A): 2.500e-08


In [124]:
print('Time to write: ', '{:.3e}'.format(float(t1-t0)))
print('Time to read: ', '{:.3e}'.format(float(t3-t2-1)))

Time to write:  2.007e-03
Time to read:  4.147e-03


In [135]:
def current(t):
    return t/1000

t = np.linspace(0, 10, 10)

t4 = time.time()

for i in range(len(t)):
    # Set the source current to the value at t[i]
    keithley.write(f':SOUR:CURR {current(t[i])}')
    time.sleep(1)
    keithley.write(f'DISPlay:USER1:TEXT "CURR: {current(t[i]):.3e} A"') #show the current being sourced. this was done due to display problems

    # Turn on the output
    keithley.write(':OUTP ON')

    # Wait for some time (optional)
    time.sleep(1)  # Adjust the sleep time as needed
t5 = time.time()

In [103]:
url = 'http://10.10.15.22/cgi-bin/ope/allch.cgi'  #url of the yokogawa containing the voltage values
html = urllib.request.urlopen(url).read() #open url and read it
soup = BeautifulSoup(html, 'html.parser') #parse the html

tables = soup.find_all('table') #find the table in the html content
table = tables[1] #there are 2 tables in the html content, we're interested in the second one (has the voltage values)

td_elements = table.find_all('td', align='center') #this block of code is getting the values of the voltage by narrwoing down on
voltage_values = []                                #some elements that their code lines have in common: having 'td', 'center', 
for td in td_elements:                             #'b', and not having 'width'. To understand it better, run 'table' to see
    b = td.find('b')                               #the full html code of the second table
    if b:
        try:
            value = float(b.text.strip())
            if 'width' not in td.attrs:
                voltage_values.append(value)
        except ValueError:
            pass
voltage_values #given in Volts 

[0.419, -0.203, 0.003, 0.0]